# A Neural Network in 11 Lines of Python

## 참고 블로그
- [11줄의 파이썬 코드로 뉴럴 네트워크 만들기(한글)](http://ddanggle.github.io/ml/ai/cs/2016/07/16/11lines.html)
- [A Neural Network in 11 Lines of Python(영어)](http://iamtrask.github.io/2015/07/12/basic-python-network/)

> 영어 원본 페이지는 링크가 깨졌는지 조회되지 않는다

## 원하는 것
|$x_0$|$x_1$|$x_2$|$y$|
|:-:|:-:|:-:|:-:|
|0|0|1|0|
|1|1|1|1|
|1|0|1|1|
|0|1|1|0|

jupyter로 렌더링된 결과가 마음에 안든다. / _ \ 테이블이 안이뻐..  
시키지도 않은 중앙정렬은 왜해..

In [1]:
import numpy as np

### Sigmoid function

원본 예제에서는 `nonlin`이 `sigmoid`함수이며 `deriv`값이 `True`일 경우에는 미분된 `sigmoid`함수로 동작한다.

In [2]:
def nonlin(x, deriv=False):
    if deriv:
        return x * (1-x)
    return 1/(1 + np.exp(-x))

밑바닥 부터 시작하는 딥러닝 책을 참고해 `sigmoid`함수를 분리하였다.  
`sigmoid_grad`라는 미분된 `sigmoid`의 함수는 해석적으로 구하였다.

In [3]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [4]:
def sigmoid_grad(x):
    return (1 - sigmoid(x)) * sigmoid(x)

### Input dataset

In [9]:
X = np.array([
    [0, 0, 1],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 1],
])
print("X.shape :", X.shape)

X.shape : (4, 3)


### Expectable output

In [8]:
Y = np.array([[0, 0, 1, 1]]).T
print("Y.shape :", Y.shape)

Y.shape : (4, 1)


### Init

In [54]:
np.random.seed(1)
W0 = 2 * np.random.random((3, 1)) - 1
print("W0.shape :", W0.shape)

W0.shape : (3, 1)


랜덤 시드를 고정해 항상 같은 값이 나오도록 한다.  
가중치를 `mean of 0` 으로 초기화

### Learning step
`X @ W0`은 `np.dot(X, W0)`과 같다.

In [55]:
for i in range(10000):
    # forward propagation
    A0 = X @ W0
    Y1 = sigmoid(A0)
    if i == 1:
        print("처음 Y1 : {}".format(Y1.T))
    
    # 예측한 것과 정답을 비교해 error를 산출
    E = Y - Y1
    delta = E * gradient_sigmoid(Y1)
    
    # 갱신
    W0 += X.T @ delta
print("최종 Y1 : {}".format(Y1.T))

처음 Y1 : [[ 0.30962857  0.42918547  0.35072407  0.47522609]]
최종 Y1 : [[  7.21568063e-04   4.80986190e-04   9.99592413e-01   9.99388522e-01]]


### Full code

In [68]:
import numpy as np

# function 선언은 생략

X = np.array([
    [0, 0, 1],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 1],
])
Y = np.array([[0, 0, 1, 1]]).T

np.random.seed(1)
def learn():
    W0 = 2 * np.random.random((3, 1)) - 1
    print("처음 Y1 : {}".format(sigmoid(X @ W0).T))
    for i in range(10000):
        # forward propagation
        A0 = X @ W0  # (4 x 3) @ (3 x 1) = (4 x 1)
        Y1 = sigmoid(A0)  # 정답 추측

        # 예측한 것과 정답을 비교해 error를 산출
        E = Y - Y1

        # 에러와 나아갈 방향을 곱한다
        delta = E * gradient_sigmoid(Y1)  # (4 x 1) * (4 x 1)은 각 요소간의 곱셈

        # 갱신
        W0 += X.T @ delta  # 학습률이 존재하지 않음
    print("최종 Y1 : {}".format(Y1.T))
    print("최종 W0 : {}".format(W0.T))

learn()

처음 Y1 : [[ 0.2689864   0.36375058  0.23762817  0.3262757 ]]
최종 Y1 : [[  7.21568063e-04   4.80986190e-04   9.99592413e-01   9.99388522e-01]]
최종 W0 : [[ 15.03841089  -0.40582887  -7.23346225]]


### 해석
`delta = E * gradient_sigmoid(Y1)`부분은 **미분에 의해 가중치가 계산된 에러**라고 생각할 수 있다.
최종 `W0`에서 보듯 `Y`를 결정하는데 $x_0$이 제일 중요하다는 것을 알 수 있다.

## 한계
웨이트를 기준으로 히든 레이어가 존재하지 않는 1층짜리 뉴럴 네트워크기 때문에 `XOR`에 대해선 동작하지 않는다.

In [69]:
X = np.array([
    [0, 0, 1],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 1],
])
Y = np.array([[0, 1, 1, 0]]).T

learn()

처음 Y1 : [[ 0.3067574   0.17919499  0.22958471  0.12818018]]
최종 Y1 : [[ 0.5  0.5  0.5  0.5]]
최종 W0 : [[ -2.35922393e-16  -2.35922393e-16   1.52655666e-16]]


그래서 중간에 히든 레이어를 추가시켜보겠다.

In [74]:
def learn2():
    # hidden layer의 노드 수를 4개로 설정했다.
    W0 = 2 * np.random.random((3, 4)) - 1
    W1 = 2 * np.random.random((4, 1)) - 1
    
    print("처음 Y2 : {}".format(sigmoid(sigmoid(X @ W0) @ W1).T))
    for i in range(10000):
        # forward propagation
        A0 = X @ W0  # (4 x 3) @ (3 x 4) = (4 x 4)
        Y1 = sigmoid(A0)  # (4 x 4)
        A1 = Y1 @ W1  # (4 x 4) @ (4 x 1) = (4 x 1)
        Y2 = sigmoid(A1)  # (4 x 1)

        # 예측한 것과 정답을 비교해 error를 산출
        E2 = Y - Y2
        if i % 1000 == 0:
            print("> 에러 E2 : {:.4f}".format(np.mean(np.abs(E2))))

        # 에러와 나아갈 방향을 곱한다
        delta2 = E2 * gradient_sigmoid(Y2)  # (4 x 1)
        
        # 이부분이 바로 역전파 부분
        # 히든레이어의 오차와 delta값을 정한다.
        E1 = delta2 @ W1.T  # (4 x 1) @ (1 x 4) = (4 x 4)
        delta1 = E1 * gradient_sigmoid(Y1)  # (4 x 4)

        # 갱신
        W1 += Y1.T @ delta2
        W0 += X.T @ delta1
    print("최종 Y2 : {}".format(Y2.T))
    print("최종 W1 : {}".format(W1.T))

learn2()

처음 Y2 : [[ 0.63541533  0.60962647  0.63330607  0.60524265]]
> 에러 E2 : 0.4994
> 에러 E2 : 0.5001
> 에러 E2 : 0.5000
> 에러 E2 : 0.4991
> 에러 E2 : 0.4991
> 에러 E2 : 0.4998
> 에러 E2 : 0.4992
> 에러 E2 : 0.4884
> 에러 E2 : 0.3610
> 에러 E2 : 0.1011
최종 Y2 : [[ 0.06745796  0.9181684   0.93220534  0.05478828]]
최종 W1 : [[ 11.77121119 -23.85579515 -15.34049799  -8.41088252]]


성공적으로 XOR문제를 예측했다.